In [1]:
import pickle
import uuid
import pandas as pd
from sklearn.pipeline import make_pipeline
import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
RUN_ID = "3742b6094a8f40558aab321accd39995"
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

mlflow.set_experiment("green-taxi-duration")

#logged_model = f"mlflow-artifacts:/1/{RUN_ID}/artifacts/model"
logged_model = f'runs:/{RUN_ID}/model'
model = mlflow.pyfunc.load_model(logged_model)

year = 2021
month = 2
taxi_type = 'green'

input_file = f'/home/adem/MLops_zoomcamp_project/web-service-mlflow/data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'/home/adem/MLops_zoomcamp_project/web-service-mlflow/batch/output/{taxi_type}/{year:04d}-{month:02d}.parquet'

In [3]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids


def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df['ride_id'] = generate_uuids(len(df))

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [4]:
def load_model(run_id):
    #logged_model = f"mlflow-artifacts:/1/{RUN_ID}/artifacts/model"
    logged_model = f'runs:/{run_id}/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    df_result.to_parquet(output_file, index=False)

In [5]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

In [6]:
!ls /home/adem/MLops_zoomcamp_project/web-service-mlflow//batch/output/green

2021-02.parquet


In [7]:
df_result = pd.read_parquet('/home/adem/MLops_zoomcamp_project/web-service-mlflow//batch/output/green/2021-02.parquet')
df_result

ride_id lpep_pickup_datetime PULocationID  \
0      eeabb5b1-1066-45fb-b38b-0d11e4c0286a  2021-02-01 00:34:03          130   
1      920e73b4-7dac-4fc2-9922-56f19310fda9  2021-02-01 00:04:00          152   
2      d3da30e7-0ed1-4bb8-95c0-853c36aeb9c4  2021-02-01 00:18:51          152   
3      dec28c1b-42ae-4b69-9279-4fa7bb3bdb8b  2021-02-01 00:53:27          152   
4      772d13c2-5630-42b6-ae31-225832f0d15e  2021-02-01 00:57:46           75   
...                                     ...                  ...          ...   
61916  810c3a9d-443f-4c7d-bffd-1fc7052cf9cf  2021-02-28 22:19:00          129   
61917  a5af9ce6-e3c7-41b1-a0cd-0979b821eb49  2021-02-28 23:18:00          116   
61918  caa9a6bd-d76d-445b-8d93-636fbbdd46e7  2021-02-28 23:44:00           74   
61919  167a7f41-6845-48dc-96dc-c4d91eacfac3  2021-02-28 23:07:00           42   
61920  344bb47f-2be3-45b2-a0f5-94754f607427  2021-02-28 23:43:00           42   

      DOLocationID  actual_duration  predicted_duration      diff  \
0              205        17.916667           18.168946 -0.252279   
1              244         6.500000            7.290331 -0.790331   
2               48        15.250000           21.278875 -6.028875   
3              241        18.233333           25.008761 -6.775428   
4               42         8.966667            9.380744 -0.414077   
...            ...              ...                 ...       ...   
61916            7        10.000000           14.549670 -4.549670   
61917          166         9.000000            9.886653 -0.886653   
61918          151        14.000000           13.597153  0.402847   
61919           42         7.000000            6.705995  0.294005   
61920           41         7.000000            8.074786 -1.074786   

                          model_version  
0      3742b6094a8f40558aab321accd39995  
1      3742b6094a8f40558aab321accd39995  
2      3742b6094a8f40558aab321accd39995  
3      3742b6094a8f40558aab321accd39995  
4      3742b6094a8f40558aab321accd39995  
...                                 ...  
61916  3742b6094a8f40558aab321accd39995  
61917  3742b6094a8f40558aab321accd39995  
61918  3742b6094a8f40558aab321accd39995  
61919  3742b6094a8f40558aab321accd39995  
61920  3742b6094a8f40558aab321accd39995  

[61921 rows x 8 columns]